# Clustering Practice - Data Scraping

This notebook will be used to source each of the datasets. I will attempt to use HTML scraping in order to extract the datasets from each postcode within South Oxfordshire.

URL for Cholsey containing OX10 9QW
http://www.ukcensusdata.com/cholsey-and-wallingford-south-e00145773/household-size-qs406ew

## Import Libraries

In [35]:
# Pandas for data manipulations
import pandas as pd

# requests and Beautiful soup for html parsing
from bs4 import BeautifulSoup
import requests


In [36]:
url = r'http://www.ukcensusdata.com/south-oxfordshire-e07000179'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE HTML>
<html>
 <head>
  <title>
   South Oxfordshire - UK Census Data 2011
  </title>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
  <meta content=" South Oxfordshire. 2011 Census Data for all census areas in England and Wales." name="description"/>
  <meta content="Good Stuff IT Services" name="author"/>
  <meta content="area" name="foo"/>
  <meta content="832605246" property="fb:admins"/>
  <meta content="B574B9C7F090DC5771DB3C1A2943B5B1" name="msvalidate.01"/>
  <meta content="YPO8ceTkDl37RPOcHU6tut7QCIEk4r9G_95t63coe_o" name="google-site-verification"/>
  <meta content="UK Census Data" property="og:site_name"/>
  <link href="/framework/main.css" rel="stylesheet" type="text/css"/>
  <link href="https://plus.google.com/106650210520079476747" rel="publisher"/>
  <link href="https://plus.google.com/106650210520079476747" rel="author"/>
  <meta content="3f9560f45b8f97bc32e8ca7aef33905d" name="fpn-

In [64]:
My_table = soup.find('select')
# print(My_table)
option_text = [x.getText() for x in My_table.find_all('option')]
option_text.pop(0)
option_text

datasets= pd.DataFrame(data=option_text, columns=['Description'])
datasets['Key'] = datasets['Description'].apply(lambda x: x.split()[-1])
datasets.to_csv('dataset_keys.csv')

# option_text[0].split()[-1]

In [17]:
url = r'http://ukcensusdata.com/a-e00145769/b-qs103ew'
website_url = requests.get(url).text
tables = pd.read_html(website_url)
tables

[                                          0      1
 0                            All Residents:  313.0
 1                     Number of households:  121.0
 2                   Average household size:    2.6
 3                  Residents in households:  313.0
 4             Residents in communal living:    0.0
 5                          Area (hectares):   12.0
 6  Population density (people per hectare):   26.6,
                        0    1
 0    All categories: Age  313
 1            Age under 1    4
 2                  Age 1    5
 3                  Age 2    5
 4                  Age 3    9
 5                  Age 4    3
 6                  Age 5    6
 7                  Age 6    2
 8                  Age 7    2
 9                  Age 8    2
 10                 Age 9    3
 11                Age 10    0
 12                Age 11    4
 13                Age 12    2
 14                Age 13    3
 15                Age 14    7
 16                Age 15    3
 17                Age 16

In [115]:
def getDataset(OA, key):
    """
    Scrapes the specified dataset from the ukcensusdata website.
    OA: Output area unique code from census
    key: Key code referring to census dataset
    
    return: dictionary of key, value pairs
    """
    
    # create the url from parameters
    url = f'http://ukcensusdata.com/a-{OA}/b-{key}'
    # request the HTML
    website_url = requests.get(url).text
    # parse the HTML
    soup = BeautifulSoup(website_url,'lxml')

    # extract the HTML referring to infotable
    table = soup.find('table', attrs={'class':"infotable"})
    # list of keys
    keys = table.find_all('td', attrs={'class':"key"})
    # list of values
    values = table.find_all('td', attrs={'class':"value"})

    # list comprehension to combine lists into key, value tuples
    keyvaluelist = [{k.getText().strip('\n\t'): v.getText().strip('\n\t')} for k,v in zip(keys, values)]
    
    return keyvaluelist



def collectDatasetsOnArea(OA, keys):
    """
    For a given area create a dictionary of the datasets given by keys
    OA: Output area unique code from census
    keys: list of dataset identifier keys
    
    return: dictionary of datasets for the area
    """
    
    datasets = {key: getDataset(OA,key) for key in keys}
    
    return datasets


def collectAllDatasets(areas, keys):
    """
    For a given list of areas and keys extract all data into a dictionary
    areas: list of OA codes
    keys: list of dataset identifier keys
    
    return: dictionary with the following heirachy: {Output_Area: {Dataset_key: Dataset}}
    """
    
    area_data_dict = [{OA: collectDatasetsOnArea(OA, keys)} for OA in areas]
    
    return area_data_dict

In [116]:
areas = ['E00145678','E00145679','E00145680']#,'E00145681','E00145682','E00145683','E00145684']
keys = ['QS416EW','QS415EW']

data = collectAllDatasets(areas, keys)

In [108]:
import json

y = json.dumps(data, indent=4)
print(y)

{
    "E00145678": {
        "QS416EW": [
            {
                "All categories: Car or van availability": "114"
            },
            {
                "No cars or vans in household": "0"
            },
            {
                "1 car or van in household": "25"
            },
            {
                "2 cars or vans in household": "54"
            },
            {
                "3 cars or vans in household": "25"
            },
            {
                "4 or more cars or vans in household": "10"
            },
            {
                "All categories: Car or van availability": "254"
            }
        ],
        "QS415EW": [
            {
                "All categories: Type of central heating in household": "114"
            },
            {
                "No central heating ": "5"
            },
            {
                "Gas central heating": "13"
            },
            {
                "Electric (including storage heaters) central 

In [120]:
from pandas.io.json import json_normalize

pd.DataFrame.from_dict(data)

,E00145678,E00145679,E00145680
0,{'QS416EW': [{'All categories: Car or van avai...,NaN,NaN
1,NaN,{'QS416EW': [{'All categories: Car or van avai...,NaN
2,NaN,NaN,{'QS416EW': [{'All categories: Car or van avai...


In [123]:
OA = 'E00145769'
key = 'QS415EW'

# create the url from parameters
url = f'http://ukcensusdata.com/a-{OA}/b-{key}'
# request the HTML
website_url = requests.get(url).text
# parse the HTML
soup = BeautifulSoup(website_url,'lxml')

# extract the HTML referring to infotable
table = soup.find('table', attrs={'class':"infotable"})
# list of keys
pd.read_html(requests.get(url).text)
# keys = table.find_all('td', attrs={'class':"key"})
# # list of values
# values = table.find_all('td', attrs={'class':"value"})

# # list comprehension to combine lists into key, value tuples
# keyvaluelist = [(k.getText().strip('\n\t'), v.getText().strip('\n\t')) for k,v in zip(keys, values)]

# keyvaluelist

[                                          0      1
 0                            All Residents:  313.0
 1                     Number of households:  121.0
 2                   Average household size:    2.6
 3                  Residents in households:  313.0
 4             Residents in communal living:    0.0
 5                          Area (hectares):   12.0
 6  Population density (people per hectare):   26.6,
                                                    0    1
 0  All categories: Type of central heating in hou...  121
 1                                 No central heating    3
 2                                Gas central heating  107
 3  Electric (including storage heaters) central h...    2
 4                                Oil central heating    5
 5  Solid fuel (for example wood, coal) central he...    0
 6                              Other central heating    1
 7               Two or more types of central heating    3]

In [88]:
mystring = '\n\t\t\tAll categories: Age\t\t\t'

mystring.strip('\n\t')

'All categories: Age'